In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def informacion(entry):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'}

    url = f"https://www.uniprot.org/uniprot/?query={entry}+Bos+taurus&sort=score"
    r = requests.get(url, headers=headers)

    soup3 =BeautifulSoup(r.text, "html.parser")
    DUDA=soup3.find_all('div',{"id":"noResultsMessage"})
        
    if DUDA == []:
        soup =BeautifulSoup(r.text, "html.parser")
        questions =soup.find_all('tr', class_='entry selected-row')
        for item in questions:
            Entry = item.find('td',{'class':'entryID'}).text
            link='https://www.uniprot.org' +item.find('td', {'class': 'entryID'}).a['href']
            Entry_names = item.find('td',{'class':''}).text

        url2= f"https://www.uniprot.org/uniprot/{Entry}"
        r2 = requests.get(url2, headers=headers)
        soup2 =BeautifulSoup(r2.text, "html.parser")

        questions2 =soup2.find_all('div', {'class':'content entry_view_content up_entry swissprot'})

        for item in questions2:
            Protein = item.find('div',{'id':'content-protein'}).text
            Gene=item.find('div',{'id':'content-gene'}).text
            Organism=item.find('div',{'id':'content-organism'}).text
            Status=item.find('a',{'title':'Reviewed (Swiss-Prot)'}).text
        
        try:
            #print(Protein, Gene, Organism, Status) 
            datos1 = [Protein, Gene, Organism, Status]
        except UnboundLocalError:
            Protein = "Not found Protein"; Gene = "Not found Gene"
            Organism = "Not found Organism"; Status = "Not found Status" 
            #datos1 = [Protein, Gene, Organism, Status]
            
        go2=soup2.find("ul",{"class":"noNumbering molecular_function"})
        if go2 == None:
            print(" ")
        else :
            goes=go2.find_all("li")

        cadena2=""
        try:
            for j in goes:
                Molecular_function=str(j.find("a").text).capitalize()  #imprime todo texto de bio_process   
                cadena3=Molecular_function+"."
                cadena2=cadena2+cadena3
        except UnboundLocalError:
            cadena2 = "Cadena2 nor found "

        go=soup2.find("ul",{"class":"noNumbering biological_process"})
        if go == None:
            print(" ")
        else :
            goes=go.find_all("li")

        cadena=""
        try:
            for j in goes:
                Biological_process_row=str(j.find("a").text).capitalize()  #imprime todo texto de bio_process   
                cadena1=Biological_process_row+"."
                cadena=cadena+cadena1
        except UnboundLocalError:
            cadena = "Cadena1 not found"
